In [1]:
import os
import sys
import time

import matplotlib.pyplot as plt
import numpy as np

import GCode
import GRBL


# Code:

Draw a 10 mm line from (0, 0) to (10, 0).

array([[ 0.,  0.],
       [10.,  0.]])

In [3]:
def VertLine(X=0, Y0=0, Yf=10, n_points=2):
    p = np.linspace(Y0, Yf, n_points, endpoint=True)
    line_points = np.array([
        X*np.ones(p.shape),
        p,
    ])
    return line_points.transpose()
VertLine()

array([[ 0.,  0.],
       [ 0., 10.]])

In [5]:
progs = list()

for n_points in range(2, 10):
    
    line_cfg = {
        "X0": 0,
        "Xf": 10,
        "Y": 0,
        "n_points": n_points
    }    
    points = HorzLine(**line_cfg)
    
    line_cfg = {
        "points": points,
        "feed":120,
        "power":128,
        "dynamic_power": True,
    }
    line = GCode.Line(points=points)
    
    
    prog_cfg={
        "lines": [line, line],
        "feed": 120
    }
    prog = GCode.Program(**prog_cfg)
    progs.append(prog)
progs

ValueError: Invalid format specifier

In [108]:
for prog in progs:
    print(len(prog.buffer))

0
0
0


In [109]:
for prog in progs:
    prog.generate_gcode()
    print(len(prog.buffer))

17
19
21


In [113]:
list(map(lambda prog: prog.generate_gcode(), progs))


[None, None, None]

In [116]:
list(map(lambda prog: len(prog.buffer), progs))

[17, 19, 21]

In [119]:
import threading

def concurrent_map(func, data):
    """
    Similar to the bultin function map(). But spawn a thread for each argument
    and apply `func` concurrently.

    Note: unlike map(), we cannot take an iterable argument. `data` should be an
    indexable sequence.
    """

    N = len(data)
    result = [None] * N

    # wrapper to dispose the result in the right slot
    def task_wrapper(i):
        result[i] = func(data[i])

    threads = [threading.Thread(target=task_wrapper, args=(i,)) for i in range(N)]
    for t in threads:
        t.start()
    for t in threads:
        t.join()

    return result

In [121]:
concurrent_map(lambda prog: prog.generate_gcode(), progs)

[None, None, None]

In [122]:
concurrent_map(lambda prog: len(prog.buffer), progs)

[17, 19, 21]

In [123]:
concurrent_map(lambda prog: prog.__repr__(), progs)

['Program<lines=2>', 'Program<lines=2>', 'Program<lines=2>']

In [128]:
concurrent_map(lambda prog: prog.dist, progs)

[40.0, 40.0, 40.0]

In [129]:
concurrent_map(lambda prog: prog.jog_dist, progs)

[20.0, 20.0, 20.0]

In [130]:
concurrent_map(lambda prog: prog.laserin_dist, progs)

[20.0, 20.0, 20.0]

In [131]:
m=concurrent_map(lambda prog: prog.laserin_dist, progs)

In [132]:
np.diff(m)

array([0., 0.])

In [135]:
np.diff(m)==0

array([ True,  True])

In [136]:
np.all(np.diff(m)==0)

True

In [138]:
assert(np.all(np.diff(m)==0))

In [139]:
flip2

array([[0, 1],
       [1, 0]])

In [143]:
reverse(None, progs[1].lines[0].points)

array([[10.,  0.],
       [ 5.,  0.],
       [ 0.,  0.]])

In [144]:
progs

[Program<lines=2>, Program<lines=2>, Program<lines=2>]